importations

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.multiclass      import OneVsOneClassifier
from sklearn.utils import resample


data preparation

In [6]:
# df = pd.read_csv('./data/urban_development_dataset.csv')
# min_class = df['development_trend_score'].value_counts().idxmin()
# print(df['development_trend_score'].value_counts())
# #undersampling 
# df_undersampled = pd.DataFrame()
# min_class_size = df['development_trend_score'].value_counts().min()
# for cls in df['development_trend_score'].unique():
#     df_class = df[df['development_trend_score'] == cls]
#     df_class_undersampled = resample(
#         df_class,
#         replace=False,  # Sample without replacement
#         n_samples=min_class_size,  # Match the size of the minority class
#         random_state=42  # Ensure reproducibility
#     )
#     df_undersampled = pd.concat([df_undersampled, df_class_undersampled])

# df_undersampled = df_undersampled.sample(frac=1, random_state=42).reset_index(drop=True)

# print(df_undersampled['development_trend_score'].value_counts())
# df = df_undersampled
# #oversampling


# #----
# print(df['development_trend_score'].value_counts())
# test_data = pd.read_csv('./data/urban_development_test_data.csv')
# common_columns = df.columns.intersection(test_data.columns)
# Y = df['development_trend_score']
# df = df[common_columns]
# test_data = test_data[common_columns]
# le = LabelEncoder()
# df.head()
# df.shape


development_trend_score
3    4019
2    2018
4    1949
1    1018
5     996
Name: count, dtype: int64
development_trend_score
5    996
1    996
2    996
4    996
3    996
Name: count, dtype: int64
development_trend_score
5    996
1    996
2    996
4    996
3    996
Name: count, dtype: int64


(4980, 20)

In [23]:
import pandas as pd
from sklearn.utils import resample
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('./data/urban_development_dataset.csv')
print("Répartition initiale des classes :")
print(df['development_trend_score'].value_counts())

#undersampling
# min_samples = df['development_trend_score'].value_counts().min()
# balanced_dfs = []
# for score in df['development_trend_score'].unique():
#     df_class = df[df['development_trend_score'] == score]
#     if len(df_class) > min_samples:
#         df_class_balanced = resample(df_class, 
#                                      replace=False, 
#                                      n_samples=min_samples, 
#                                      random_state=42)
#     else:
#         df_class_balanced = resample(df_class, 
#                                      replace=True, 
#                                      n_samples=min_samples, 
#                                      random_state=42)
#     balanced_dfs.append(df_class_balanced)
# df = pd.concat(balanced_dfs)
#oversampling
max_class_count = df['development_trend_score'].value_counts().max()
dfs = []
for label in df['development_trend_score'].unique():
    df_label = df[df['development_trend_score'] == label]
    
    # Sur-échantillonner la classe pour atteindre le nombre max_class_count
    df_label_upsampled = resample(
        df_label,
        replace=True,              # Autoriser les doublons
        n_samples=max_class_count, # Nombre d'échantillons après sur-échantillonnage
        random_state=42            # Reproductibilité
    )
    
    dfs.append(df_label_upsampled)
df_balanced = pd.concat(dfs)
df = df_balanced

print("\nRépartition après équilibrage :")
print(df['development_trend_score'].value_counts())

test_data = pd.read_csv('./data/urban_development_test_data.csv')
common_columns = df.columns.intersection(test_data.columns)

Y = df['development_trend_score']
df = df[common_columns]
test_data = test_data[common_columns]

le = LabelEncoder()
df.head()


Répartition initiale des classes :
development_trend_score
3    4019
2    2018
4    1949
1    1018
5     996
Name: count, dtype: int64

Répartition après équilibrage :
development_trend_score
4    4019
5    4019
2    4019
1    4019
3    4019
Name: count, dtype: int64


,population_density,median_age,average_household_income,number_of_schools,number_of_hospitals,number_of_parks,public_transport_availability,unemployment_rate,retail_density,office_space_availability,green_space_percentage,average_temperature,air_quality_index,crime_rate_per_1000,fire_station_proximity,proximity_to_highways,internet_speed,housing_price_index,year,road_density
5846,10854.540185,28.768197,85199.479760,3.0,1.0,3.0,0.968911,6.358546,42.998103,47808.665014,30.073580,10.113789,52.830152,19.879518,0.669529,18.877326,91.490016,88.356512,2022.0,108.759141
7544,7116.340188,19.603743,37137.715368,7.0,3.0,6.0,0.742236,4.228577,30.537517,66341.178029,34.961826,13.203461,86.298477,65.531645,6.284401,11.449390,31.791613,107.527199,2022.0,70.697805
4545,6168.628333,19.555451,63351.767633,9.0,0.0,0.0,NaN,4.145707,45.437759,31054.949220,46.106749,16.503922,56.146415,31.978132,3.830554,9.470108,61.333742,98.681242,2018.0,64.243977
6680,4369.467332,38.933783,54554.097575,7.0,3.0,5.0,0.642622,5.976038,47.733647,27306.192868,21.120700,22.609254,31.514748,18.892706,7.960523,3.198202,35.004245,106.552954,2018.0,NaN
5855,1403.570521,38.234524,51493.071701,3.0,3.0,1.0,0.171976,4.528690,46.886549,16408.720240,43.017724,11.756975,39.365717,28.556242,5.401078,17.167561,73.958336,87.267601,2021.0,16.100594


Preprocessing data using median for numerical features | KNN


In [24]:
from sklearn.impute import KNNImputer
numerical_features = [
    "population_density", 'median_age', 'average_household_income', 
    'number_of_schools', 'number_of_hospitals', 'number_of_parks', 
    'public_transport_availability', 'unemployment_rate', 'retail_density', 
    'office_space_availability', 'green_space_percentage', 'average_temperature', 
    'air_quality_index', 'crime_rate_per_1000', 'fire_station_proximity', 
    'proximity_to_highways', 'internet_speed', 'road_density', 
    'housing_price_index', 'year'
]
# imputer = SimpleImputer(strategy='median') 
# numerical_features = ["population_density",'median_age', 'average_household_income', 'number_of_schools', 'number_of_hospitals','number_of_parks','public_transport_availability' ,'unemployment_rate', 'retail_density','office_space_availability','green_space_percentage','average_temperature','air_quality_index','crime_rate_per_1000','fire_station_proximity','proximity_to_highways','internet_speed','road_density','housing_price_index','year',  ]
# df[numerical_features] = imputer.fit_transform(df[numerical_features])
# df.isnull().sum()


knn_imputer = KNNImputer(n_neighbors=5)  
df[numerical_features] = knn_imputer.fit_transform(df[numerical_features])
print(df.isnull().sum())


population_density               0
median_age                       0
average_household_income         0
number_of_schools                0
number_of_hospitals              0
number_of_parks                  0
public_transport_availability    0
unemployment_rate                0
retail_density                   0
office_space_availability        0
green_space_percentage           0
average_temperature              0
air_quality_index                0
crime_rate_per_1000              0
fire_station_proximity           0
proximity_to_highways            0
internet_speed                   0
housing_price_index              0
year                             0
road_density                     0
dtype: int64


/tmp/ipykernel_6443/1268813566.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[numerical_features] = knn_imputer.fit_transform(df[numerical_features])


Normalisation

In [25]:
# correlation_matrix = df.corr()  
# threshold = 0.9
# correlation_matrix
# high_corr_features = set()
# for i in range(len(correlation_matrix.columns)):
#     for j in range(i):
#         if abs(correlation_matrix.iloc[i, j]) > threshold:
#             colname = correlation_matrix.columns[i]
#             high_corr_features.add(colname)

# print(f"Features to drop due to high correlation: {high_corr_features}")
# df = df.drop(high_corr_features, axis=1)

In [26]:
X = df
scaler = StandardScaler()
X = scaler.fit_transform(X)

check outliers

In [87]:
from scipy.stats import zscore
z_scores = np.abs(zscore(df)) 
outlier_mask = (z_scores > 3).any(axis=1)
plt.figure(figsize=(10, 6))
print(z_scores)

      population_density  median_age  average_household_income  \
0               0.512888    0.756167                  0.029903   
1               0.137358    0.528189                  0.323029   
2               0.667493    0.669652                  0.409018   
3               1.563880    0.085279                  0.108920   
4               0.235552    1.244407                  0.598128   
...                  ...         ...                       ...   
9995            1.336617    0.381161                  0.286295   
9996            2.042161    1.436593                  0.404432   
9997            0.718044    0.371057                  0.260359   
9998            0.511917    1.795142                  0.735555   
9999            0.664113    1.760282                  0.167272   

      number_of_schools  number_of_hospitals  number_of_parks  \
0              0.006087             0.008146         0.031991   
1              0.452271             1.451696         0.031991   
2           

<Figure size 1000x600 with 0 Axes>

In [40]:
# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.decomposition import PCA

# pca = PCA(n_components=2)
# X_pca = pca.fit_transform(X)

# loadings = pca.components_.T * np.sqrt(pca.explained_variance_)
# plt.figure(figsize=(12, 8))

# plt.scatter(X_pca[:, 0], X_pca[:, 1], c=Y, cmap='viridis', edgecolor='k', s=50)
# plt.colorbar(label='Development Trend Score')
# plt.title('PCA Biplot')
# plt.xlabel('PC1')
# plt.ylabel('PC2')

# for i, var in enumerate(X.T):
#     plt.arrow(0, 0, loadings[i, 0], loadings[i, 1], color='r', alpha=0.7, head_width=0.05, linewidth=1.5)
#     plt.text(loadings[i, 0] * 1.1, loadings[i, 1] * 1.1, var, color='r', ha='center', va='center', fontsize=10)

# plt.grid()
# plt.show()


split data

In [27]:
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.2, random_state=42)
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)
# train_x, train_y = X, Y


(16076, 20) (4019, 20) (16076,) (4019,)


In [28]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier


model = svm.SVC(kernel='linear', gamma='scale' , C=0.1)
model.fit(X,Y)

ovo_model = OneVsOneClassifier(model)
ovo_model.fit(train_x, train_y)
y_pred = ovo_model.predict(test_x)

# Evaluate the model
print(f"Test Accuracy: {accuracy_score(test_y, y_pred)}")
print(f"Classification Report for SVM:\n{classification_report(test_y, y_pred)}")

Test Accuracy: 0.221945757651157
Classification Report for SVM:
              precision    recall  f1-score   support

           1       0.23      0.28      0.25       803
           2       0.22      0.24      0.23       774
           3       0.21      0.13      0.16       827
           4       0.22      0.25      0.23       777
           5       0.23      0.22      0.23       838

    accuracy                           0.22      4019
   macro avg       0.22      0.22      0.22      4019
weighted avg       0.22      0.22      0.22      4019



In [29]:
svc_model = svm.SVC(kernel='linear', gamma='scale' , C=0.1)
svc_model.fit(train_x, train_y)
y_pred = svc_model.predict(test_x)
print(f"Test Accuracy: {accuracy_score(test_y, y_pred)}")
print(f"Classification Report for SVM:\n{classification_report(test_y, y_pred)}")

Test Accuracy: 0.22045284896740483
Classification Report for SVM:
              precision    recall  f1-score   support

           1       0.23      0.28      0.25       803
           2       0.22      0.25      0.23       774
           3       0.21      0.13      0.16       827
           4       0.22      0.24      0.23       777
           5       0.22      0.21      0.22       838

    accuracy                           0.22      4019
   macro avg       0.22      0.22      0.22      4019
weighted avg       0.22      0.22      0.22      4019



testing data

In [30]:
from sklearn.impute import KNNImputer
# test_data = pd.read_csv('./data/urban_development_test_data.csv')
# test_data.shape

numerical_features = [
    "population_density", 'median_age', 'average_household_income', 
    'number_of_schools', 'number_of_hospitals', 'number_of_parks', 
    'public_transport_availability', 'unemployment_rate', 'retail_density', 
    'office_space_availability', 'green_space_percentage', 'average_temperature', 
    'air_quality_index', 'crime_rate_per_1000', 'fire_station_proximity', 
    'proximity_to_highways', 'internet_speed', 'road_density', 
    'housing_price_index', 'year'
]
# imputer = SimpleImputer(strategy='median') 
# test_data[numerical_features] = imputer.fit_transform(test_data[numerical_features])
# test_data.isnull().sum()
knn_imputer = KNNImputer(n_neighbors=5)  
test_data[numerical_features] = knn_imputer.fit_transform(test_data[numerical_features])
# print(test_data.isnull().sum())


output

In [71]:
output = ovo_model.predict(test_data)
output
submission = pd.DataFrame({
    "ID": test_data.index + 1,  
    "development_trend_score": output
})
submission_file_path = './test/development_trend_predictions_classification.csv'
submission.to_csv(submission_file_path, index=False)
print(f"Submission file saved at: {submission_file_path}")

/home/self-ouss/notebooks/.venv/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but OneVsOneClassifier was fitted without feature names
  warnings.warn(


Submission file saved at: ./test/development_trend_predictions_classification.csv


grid search for svm

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
print(f"Performing Cross-Validation for SVM")

param_grid = {
'C': [0.1, 1, 10],  
'kernel': ['linear', 'rbf'],  
'gamma': ['scale', 'auto']  
}
model = svm.SVC()
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(train_x, train_y)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Cross-Validation Score: {grid_search.best_score_}")

best_model = grid_search.best_estimator_
y_pred = best_model.predict(test_x)

# Evaluate the model
print(f"Test Accuracy: {accuracy_score(test_y, y_pred)}")
print(f"Classification Report for SVM:\n{classification_report(test_y, y_pred)}")


grid search for randomforest

In [31]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV


param_grid_rf = {
    'n_estimators': [100, 200],   
    'max_depth': [None, 10, 20],      
    'min_samples_split': [5, 10],  
    'min_samples_leaf': [2, 4],    
}

rf_model = RandomForestClassifier(random_state=42)

grid_search_rf = GridSearchCV(
    estimator=rf_model,
    param_grid=param_grid_rf,
    cv=5, 
    scoring='accuracy', 
    verbose=1,
    n_jobs=-1  
)

grid_search_rf.fit(train_x, train_y)

print(f"Best Parameters for Random Forest: {grid_search_rf.best_params_}")
print(f"Best Cross-Validation Score for Random Forest: {grid_search_rf.best_score_}")

best_rf_model = grid_search_rf.best_estimator_
y_pred_rf = best_rf_model.predict(test_x)

# print(f"Test Accuracy for Random Forest: {accuracy_score(test_y, y_pred_rf)}")
# print(f"Classification Report for Random Forest:\n{classification_report(test_y, y_pred_rf)}")


Fitting 5 folds for each of 24 candidates, totalling 120 fits


Best Parameters for Random Forest: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 100}
Best Cross-Validation Score for Random Forest: 0.40149999999999997


In [34]:


from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier

# rfm = RandomForestClassifier(max_depth=10, min_samples_leaf=4, min_samples_split=5, n_estimators=100, random_state=42)
rfm = RandomForestClassifier(random_state=42, n_estimators=200)
# rfm.fit(X, Y)
rfm.fit(train_x, train_y)
y_pred_rf = rfm.predict(test_x)

print(f"Test Accuracy for Random Forest: {accuracy_score(test_y, y_pred_rf)}")
print(f"Classification Report for Random Forest:\n{classification_report(test_y, y_pred_rf)}")


Test Accuracy for Random Forest: 0.8882806668325454
Classification Report for Random Forest:
              precision    recall  f1-score   support

           1       0.98      0.97      0.98       803
           2       0.88      0.84      0.86       774
           3       0.75      0.84      0.79       827
           4       0.87      0.85      0.86       777
           5       0.98      0.94      0.96       838

    accuracy                           0.89      4019
   macro avg       0.89      0.89      0.89      4019
weighted avg       0.89      0.89      0.89      4019



In [35]:
output = rfm.predict(test_data)
submission = pd.DataFrame({
    "ID": test_data.index + 1,  
    "development_trend_score": output
})
submission_file_path = './test/development_trend_predictions_classification.csv'
submission.to_csv(submission_file_path, index=False)
print(f"Submission file saved at: {submission_file_path}")
output[:200]

Submission file saved at: ./test/development_trend_predictions_classification.csv


/home/self-ouss/notebooks/.venv/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


array([4, 4, 1, 3, 5, 5, 2, 5, 5, 4, 2, 5, 4, 4, 2, 5, 4, 5, 5, 2, 5, 5,
       2, 4, 4, 3, 5, 4, 4, 5, 5, 3, 5, 4, 5, 4, 5, 3, 2, 2, 5, 4, 4, 2,
       4, 5, 3, 2, 5, 5, 5, 2, 5, 5, 4, 4, 5, 4, 5, 5, 4, 5, 3, 4, 4, 5,
       5, 4, 4, 5, 3, 4, 5, 5, 5, 5, 4, 5, 3, 3, 4, 5, 5, 2, 5, 5, 5, 4,
       2, 5, 5, 5, 5, 5, 4, 5, 5, 4, 5, 5, 4, 2, 5, 5, 5, 5, 5, 5, 2, 5,
       4, 3, 3, 5, 5, 5, 5, 5, 4, 5, 4, 2, 4, 5, 4, 4, 5, 5, 2, 2, 5, 3,
       4, 5, 5, 4, 5, 5, 5, 5, 5, 2, 2, 5, 4, 4, 4, 3, 5, 4, 3, 4, 5, 5,
       4, 2, 4, 5, 5, 5, 5, 5, 4, 4, 5, 4, 5, 5, 5, 5, 5, 4, 4, 5, 3, 5,
       2, 4, 4, 2, 5, 2, 4, 5, 5, 1, 5, 5, 5, 5, 3, 4, 5, 2, 4, 5, 3, 3,
       5, 5])